<a href="https://colab.research.google.com/github/pinnouse/zeroshot-unsurpervised-mt/blob/main/CSC413_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install apache_beam mwparserfromhell
!pip install transformers
!pip install datasets
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

# multilingual CLIP pretrained
# https://github.com/FreddeFrallan/Multilingual-CLIP
!pip install multilingual-clip
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dill-0.3.1.1-py3-none-any.whl
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.14 requires dill>=0.3.6, but you have dill 0.3.1.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.1.1
    Uninstalling dill-0.3.1.1:
      Successfully uninstalled dill-0.3.1.1
ERROR: pip's dependency resolver does not cur

# Data Processing

In [2]:
from datasets import load_dataset

dataset = load_dataset("wikipedia", "20220301.simple")
dataset_fr = load_dataset("wikipedia", "20220301.fr")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [4]:
text = map(lambda x: x['text'].replace("\n", ' ').split(". "), dataset['train'])
text_long = []
for t in text:
  for s in t:
    text_long.append(s) 

In [5]:
text_fr = map(lambda x: x['text'].replace("\n", ' ').split(". "), dataset_fr['train'])
for t in text_fr:
  print(t)
  break

['Paul Jules Antoine Meillet, né le  à Moulins (Allier) et mort le  à Châteaumeillant (Cher), est le principal linguiste français des premières décennies du ', 'Il est aussi philologue', " Biographie  D'origine bourbonnaise, fils d'un notaire de Châteaumeillant (Cher), Antoine Meillet fait ses études secondaires au lycée de Moulins", " Étudiant à la faculté des lettres de Paris à partir de 1885 où il suit notamment les cours de Louis Havet, il assiste également à ceux de Michel Bréal au Collège de France et de Ferdinand de Saussure à l'École pratique des hautes études", " En 1889, il est major de l'agrégation de grammaire", " Il assure à la suite de Saussure le cours de grammaire comparée, qu'il complète à partir de 1894 par une conférence sur les langues persanes", " En 1897, il soutient sa thèse pour le doctorat ès lettres (Recherches sur l'emploi du génitif-accusatif en vieux-slave)", "En 1905, il occupe la chaire de grammaire comparée au Collège de France, où il consacre ses cours 

In [6]:
import torch
import torchtext
import clip
import numpy as np
from sentence_transformers import SentenceTransformer, util

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

text_model = SentenceTransformer('sentence-transformers/clip-ViT-B-32-multilingual-v1')

context_length = 64

#glove = torchtext.vocab.GloVe(name="6B", dim=50)
#ft = torchtext.vocab.FastText(language="simple")

## Tokenize/Embbed English data

In [7]:
train_data_eng = []
for (i, t) in enumerate(text_long):
  if i > 500:
    break
  tokenized = tokenizer(t, padding='max_length', max_length=64, return_tensors='pt').input_ids #took 18 mins to run
  if len(tokenized) <= 64:
    clip_emb = text_model.encode(t)
    train_data_eng.append((t,clip_emb,tokenized[0]))

## Tokenize/Embbed French data

In [8]:
train_data_fr = []

for (i, t) in enumerate(text_fr):
  if i > 100:
    break
  sentence = t[0]
  if len(sentence.split(' ')) > 60:
    continue
  tokenized = tokenizer(sentence, padding='max_length', max_length=64, return_tensors='pt')['input_ids']
  # tokenized = tokenized.to(device)
  train_data_fr.append((sentence, tokenized[0]))

# GAN+Autoencoder Idea
![Diagram](https://media.discordapp.net/attachments/1085342382679138385/1090810686793318491/image.png?width=1440&height=592)
Orange represents the zeroshot translation task

## Define Models

We will create 4 models:
1. Monolingual generator (expert at i.e. FR, DE) - transformer encoder+decoder
2. CLIP Embedding Decoder - transformer decoders
3. Discriminator - linear neural network (subject to change)
4. Translator - linear model from embedding space to embedding space (thought of as a transform function from FR -> EN)

We refer to 2 agents that are learning to understand their respective languages, both represented by transformers.
`Base expert` is the agent that 'understands' English (from CLIP).
`Other expert` is agent that is an expert of the other language.

In [33]:
from torch import nn
from torch import Tensor
import math

# https://pytorch.org/tutorials/beginner/transformer_tutorial.html
class PositionalEncoding(nn.Module):
  def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
    super().__init__()
    self.dropout = nn.Dropout(p=dropout)

    position = torch.arange(max_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
    pe = torch.zeros(max_len, 1, d_model)
    pe[:, 0, 0::2] = torch.sin(position * div_term)
    pe[:, 0, 1::2] = torch.cos(position * div_term)
    self.register_buffer('pe', pe)

  def forward(self, x: Tensor) -> Tensor:
    """
    Args:
        x: Tensor, shape [seq_len, batch_size, embedding_dim]
    """
    x = x + self.pe[:x.size(0)]
    return self.dropout(x)

# https://jamesmccaffrey.wordpress.com/2022/09/09/simplest-transformer-seq-to-seq-example/
class Transformer(nn.Module): # same language/monolingual
  def __init__(self, ntoken: int, d_model: int = 512, nhead: int = 4, d_hid: int = 512,
               nlayers: int = 4, dropout: float = 0.5):
    super().__init__()

    self.model_type = 'Transformer'

    self.embed = nn.Embedding(ntoken, d_model)

    self.pos_enc = PositionalEncoding(d_model, dropout)
    encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, d_hid, dropout,
                                                batch_first=True)
    self.encoder = nn.TransformerEncoder(encoder_layers, nlayers)

    decoder_layers = nn.TransformerDecoderLayer(d_model, nhead, d_hid, dropout,
                                                batch_first=True)
    self.decoder = nn.TransformerDecoder(decoder_layers, nlayers)

    self.d_model = d_model

    self.dense = nn.Linear(d_model, ntoken)

    self.init_weights()

  def init_weights(self) -> None:
    initrange = 0.1
    self.dense.weight.data.uniform_(-initrange, initrange)
    self.dense.bias.data.zero_()

  def forward(self, src, tgt, tgt_mask, tp_mask=None, sp_mask=None):
    s = self.embed(src)
    t = self.embed(tgt)

    s = self.pos_enc(s)
    t = self.pos_enc(t)

    e = self.encoder(s, src_key_padding_mask=sp_mask)
    d = self.decoder(t, e, tgt_mask=tgt_mask, tgt_key_padding_mask=tp_mask, memory_key_padding_mask=sp_mask)
    z = self.dense(d)
    # z = nn.functional.softmax(z, dim=2)
    return z, e

class Translator(nn.Module):
  def __init__(self, i_embed_size = 512, o_embed_size = 512, nlayers = 1, hidden = 1024):
    super().__init__()
    
    # based off https://arxiv.org/abs/1809.03633
    self.G = nn.Sequential(nn.Linear(i_embed_size, hidden))
    self.F = nn.Sequential(nn.Linear(o_embed_size, hidden))
    for i in range(nlayers-1):
      self.G.append(nn.Linear(hidden, hidden))
      self.F.append(nn.Linear(hidden, hidden))
    self.G.append(nn.Linear(hidden, o_embed_size))
    self.F.append(nn.Linear(hidden, i_embed_size))

  def forward(self, i_emb):
    o = self.G(i_emb)
    i = self.F(o)
    return o, i
  
class Decoder(nn.Module):
  def __init__(self, ntoken: int, d_model: int = 512, nhead: int = 4, d_hid: int = 512,
               dropout: float = 0.5, nlayers: int = 4):
    super().__init__()
    self.embed = nn.Embedding(ntoken, d_model)
    self.pos_encoder = PositionalEncoding(d_model, dropout)

    decoder_layers = nn.TransformerDecoderLayer(d_model, nhead, d_hid, dropout)
    self.decoder = nn.TransformerDecoder(decoder_layers, nlayers)
  
  def forward(self, enc, tgt, tgt_mask):
    t = self.embed(tgt)
    t = self.pos_enc(t)

    d = self.decoder(tgt, enc, tgt_mask=tgt_mask)
    return d
    
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(500, 100),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(100, 1),
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        out = self.model(x)
        return out.view(x.size(0))

## Train Loop

PyTorch training for GAN:
[PyTorch blog](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html)

In [39]:
from torch import optim
# https://huggingface.co/docs/transformers/main_classes/optimizer_schedules#transformers.Adafactor
from transformers.optimization import Adafactor
import matplotlib.pyplot as plt

transformer = Transformer(tokenizer.vocab_size) # temp vocab size
translate = Translator(hidden=512)

context_length = 64

# TODO: add device specific code

def pad(tokens, context_length):
  l = list(tokens)
  for _ in range(context_length - len(tokens)):
    l.append(tokenizer.pad_token_id)
  return np.array(l)

def train(real_decoder, transformer, discriminator, translate, # our four models
          real_train, other_train, real_valid = None, other_valid = None,
          epochs = 10, batch_size = 256):
  batch_data = []

  data_loader = [(real_train, other_train)]

  criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
  mse = nn.MSELoss()

  # r_optim = optim.RAdam(real_decoder.parameters())
  # g_optim = optim.RAdam(generator.parameters())
  # r_optim = Adafactor(real_decoder.parameters())
  g_optim = Adafactor(transformer.parameters())
  t_optim = Adafactor(translate.parameters())
  # d_optim = optim.Adam(discriminator.parameters())

  r_iterations = len(real_train) // batch_size
  o_iterations = len(other_train) // batch_size
  # r_iterations = real_train.shape[0] // batch_size
  # o_iterations = other_train.shape[0] // batch_size

  n = min(r_iterations, o_iterations)

  losses = []

  for e in range(epochs):
    epoch_loss = 0
    print(f'Epoch {e+1}:')
    for i, (r_x, o_x) in enumerate(data_loader):
      # r_x: (english sentence: str, CLIP embeddings: float[512], tokens: one-hots[num tokens])
      # o_x:
      # if r_x.shape[0] % batch_size != 0: # not full batch
      #   break
      if (i + 1) % 100 == 0:
        print(f'Iteration {i+1} of {n}')

      # learn decoder
      # r_optim.zero_grad()
      # ...

      # "other" generator self learning
      # https://jamesmccaffrey.wordpress.com/2022/09/09/simplest-transformer-seq-to-seq-example/
      xx = list(map(lambda x: x[1].numpy(), o_x))
      xx = torch.tensor(np.array(xx))
      
      src = xx
      tgt = src
      tgt_in = tgt[:,:-1]
      tgt_expect = tgt[:,1:]
      t_mask = nn.Transformer.generate_square_subsequent_mask(context_length - 1)
      # https://pytorch.org/tutorials/beginner/translation_transformer.html#seq2seq-network-using-transformer
      tgt_attn_mask = (tgt_in == tokenizer.pad_token_id)
      attn_mask = (src == tokenizer.pad_token_id)

      output, other_embeddings = transformer(src, tgt_in, tgt_mask=t_mask, tp_mask=tgt_attn_mask, sp_mask=attn_mask) # [bs,seq,vocab]

      # get preds shape to conform to tgt_expect
      output = output.permute(0,2,1)  # now [bs, vocab, seq]

      loss = criterion(output, tgt_expect)
      epoch_loss += loss.item()

      g_optim.zero_grad()
      loss.backward()
      g_optim.step()
    print(f'\ttrain loss: {epoch_loss}')
    losses.append(epoch_loss)
  # plt.subplot(0)
  plt.xticks(np.arange(epochs))
  plt.plot(epoch_loss)
  plt.show()

print(train_data_fr[0])
train(None, transformer, None, translate, [], [train_data_fr[0]] * 16, epochs=100)

("L’algèbre linéaire est la branche des mathématiques qui s'intéresse aux espaces vectoriels et aux transformations linéaires, formalisation générale des théories des systèmes d'équations linéaires", tensor([  101,   149,   100, 10164, 10240, 13340, 13724, 11614, 11246, 16556,
        10176, 10109, 58760, 10139, 71777, 10355,   187,   112, 26391, 20017,
        12818, 10754, 84355, 46514, 19428, 10107, 10131, 10754, 44510, 10107,
        11614, 11246, 29194,   117, 23129, 20312, 28274, 10139, 38914, 10107,
        10139, 48273,   172,   112,   263, 32973, 15024, 11614, 11246, 29194,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0]))
Epoch 1:


/usr/local/lib/python3.9/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


	train loss: 12.529891014099121
Epoch 2:
	train loss: 10.283712387084961
Epoch 3:
	train loss: 8.806689262390137
Epoch 4:
	train loss: 8.091115951538086
Epoch 5:
	train loss: 7.325162887573242
Epoch 6:
	train loss: 6.720075607299805
Epoch 7:
	train loss: 6.2231764793396
Epoch 8:
	train loss: 5.82277774810791
Epoch 9:
	train loss: 5.524688720703125
Epoch 10:
	train loss: 5.360691547393799
Epoch 11:
	train loss: 5.171264171600342
Epoch 12:
	train loss: 4.5983662605285645
Epoch 13:
	train loss: 4.376806735992432
Epoch 14:
	train loss: 4.357534408569336
Epoch 15:
	train loss: 3.7334892749786377
Epoch 16:
	train loss: 3.593440055847168
Epoch 17:
	train loss: 3.371273994445801
Epoch 18:
	train loss: 3.5296051502227783
Epoch 19:
	train loss: 3.333890676498413
Epoch 20:
	train loss: 3.0240259170532227
Epoch 21:
	train loss: 2.374063491821289
Epoch 22:
	train loss: 2.288400888442993
Epoch 23:
	train loss: 2.0318851470947266
Epoch 24:
	train loss: 1.6393728256225586
Epoch 25:
	train loss: 1.4813

TypeError: ignored

<Figure size 640x480 with 0 Axes>

In [41]:
t = pad(train_data_fr[0][1], context_length)
src = torch.tensor(np.array([t]))
print(src.shape)
print(t)
# should predict 5,6,7,8,7,6,5

with torch.no_grad():
  s = list(t[0:1])
  for i in range(30):
    tgt_in = torch.tensor([s])
    t_mask = nn.Transformer.generate_square_subsequent_mask(len(s))
    tgt_attn_mask = (tgt_in == tokenizer.pad_token_id)
    attn_mask = src == tokenizer.pad_token_id
    # print(tgt_in)
    preds, emb = transformer(src, tgt_in, tgt_mask=t_mask, tp_mask=tgt_attn_mask, sp_mask=attn_mask)
    m = torch.argmax(preds, dim=2)
    print(m)
    tk = m[0][-1]
    if tk == tokenizer.eos_token_id:
      break
    word = tokenizer.decode(tk)
    print(word)
    s.append(tk)
  print(tokenizer.decode(s))
# result is 12 logits where largest is at the
# predicted token

torch.Size([1, 64])
[  101   149   100 10164 10240 13340 13724 11614 11246 16556 10176 10109
 58760 10139 71777 10355   187   112 26391 20017 12818 10754 84355 46514
 19428 10107 10131 10754 44510 10107 11614 11246 29194   117 23129 20312
 28274 10139 38914 10107 10139 48273   172   112   263 32973 15024 11614
 11246 29194   102     0     0     0     0     0     0     0     0     0
     0     0     0     0]
tensor([[149]])
L
tensor([[149, 100]])
[ U N K ]
tensor([[  149,   100, 10164]])
a l
tensor([[  149,   100, 10164, 10240]])
# # g
tensor([[  149,   100, 10164, 10240, 13340]])
# # è
tensor([[  149,   100, 10164, 10240, 13340, 13724]])
# # b r e
tensor([[  149,   100, 10164, 10240, 13340, 13724, 11614]])
l i
tensor([[  149,   100, 10164, 10240, 13340, 13724, 11614, 11246]])
# # n é
tensor([[  149,   100, 10164, 10240, 13340, 13724, 11614, 11246, 16556]])
# # a i r e
tensor([[  149,   100, 10164, 10240, 13340, 13724, 11614, 11246, 16556, 10176]])
e s t
tensor([[  149,   100, 10164, 10